# `ln.Session` and lazily loading relationships

In [ ]:
import lamindb as ln
import lamindb.schema as lns
import pytest
from sqlalchemy.orm.exc import DetachedInstanceError

ln.nb.header()

Let's create related sample data records and add them to the database:

In [ ]:
pipeline = lns.Pipeline(name="Transform A")
run = lns.Run(name="Solve Problem X", pipeline=pipeline)

In [ ]:
run.pipeline

In [ ]:
ln.add(run)

Both records got just added to the database.

In the background, a `Session` object was created, which connected to the database, inserted the records, and closed the connection.

## Query results without session

In [ ]:
run_queried = ln.select(lns.Run, name="Solve Problem X").first()

Also here, in the background, a session was created and closed. This is good enough if we need to use simple properties of the returned record, for instance, the pipeline id:

In [ ]:
run_queried

In [ ]:
run_queried.pipeline_id

However, if we'd like to access the entire related record (here, the `Pipeline`), we'll get a `DetachedInstanceError` error (it would tell us that the "lazy load operation of attribute 'pipeline' cannot proceed"):

In [ ]:
with pytest.raises(DetachedInstanceError):
    run_queried.pipeline

The queried run would need to have an open connection to the DB in order for it to automatically load the related record. Under the hood, it needs to perform an automated query for this.

But when `ln.select(...).first()` completed its execution, the database connection was closed.

## The Session object

In order to lazily load related data records, we need to use a `Session` object!

In [ ]:
ss = ln.Session()

The `Session` object comes with `add`, `delete` and `select`, just as the global namespace. They are equivalent to the global version, with the only difference being that all data records manipulated will be bound to an open session.

In [ ]:
run_session = ss.select(lns.Run, name="Solve Problem X").first()

In [ ]:
run_session

It's clear we don't need it for the simple attributes. But we need it for lazily loaded relationships:

In [ ]:
run_session.pipeline

Let us close the session.

In [ ]:
ss.close()

Given we already loaded the pipeline record, it's still available in memory.

In [ ]:
run_session.pipeline

But, we can't access the `notebook` relationship, as the session is now closed.

In [ ]:
with pytest.raises(DetachedInstanceError):
    run_session.notebook

## The Session object in a context manager

We can also call `Session` in a context manager:

In [ ]:
with ln.Session() as ss:
    run_session2 = ss.select(lns.Run, name="Solve Problem X").first()
    print(run_session2.pipeline)

In [ ]:
run_session2.pipeline

In [ ]:
with pytest.raises(DetachedInstanceError):
    run_session2.notebook